In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [5]:
# The article header
news_title = soup.find('div', class_="content_title").text
print(news_title)



NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network




In [6]:
news_synopsis = soup.find('div', class_='rollover_description_inner').text
print(news_synopsis)


Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.



In [7]:
#for row in soup.find_all('div',attrs={"class" : "article_teaser_body"}):
#    print(row)

In [8]:
#Groundwork for splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
pic_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(pic_url)

In [10]:
html = browser.html
pic_soup = BeautifulSoup(html, 'html.parser')

picture = pic_soup.find('article').find('a').get('data-fancybox-href')

In [11]:
print(picture)

/spaceimages/images/mediumsize/PIA16726_ip.jpg


In [12]:
modified_url = pic_url.split('?')[0]
modified_url

'https://www.jpl.nasa.gov/spaceimages/'

In [13]:
all_content = {}
modified_picture = picture.split('/')[2]+"/"+picture.split('/')[3]+"/"+picture.split('/')[4]
new_url = modified_url + modified_picture
all_content["featured_image"] = new_url

In [14]:
print(all_content["featured_image"])

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16726_ip.jpg


In [15]:
# Twitter URL of weather to be scraped
twitter_url = 'https://twitter.com/marswxreport?lang=en'
# Retrieve page with the requests module
response = requests.get(twitter_url)
time.sleep(1)
# Create BeautifulSoup object; parse with 'lxml'
twitter_soup = BeautifulSoup(response.text, 'html.parser')
timeline = twitter_soup.select('#timeline li.stream-item')

In [16]:
#print(timeline)

In [17]:
all_tweets = []
for tweet in timeline:
    #tweet_id = timeline['data-item-id']
    tweet_text = tweet.select('p.tweet-text')[0].get_text()
    all_tweets.append({"text": tweet_text})

mars_weather = all_tweets[0]
print(mars_weather)

{'text': 'InSight sol 450 (2020-03-02) low -93.5ºC (-136.4ºF) high -10.4ºC (13.3ºF)\nwinds from the SSW at 5.5 m/s (12.4 mph) gusting to 20.6 m/s (46.1 mph)\npressure at 6.30 hPapic.twitter.com/82lzRqibcC'}


In [18]:
#Mars facts scrape
fact_url = 'https://space-facts.com/mars/'
facts_tables = pd.read_html(fact_url)
facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [19]:
facts_df = facts_tables[0]
facts_df.columns = ['Description', 'Value']
facts_df.set_index('Description', inplace=True)
facts_df.head()

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [20]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)
hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html, 'html.parser')

In [25]:
hemisphere_title_clean = []
url_component = []
all_hemispheres = hemisphere_soup.find_all('a', class_='itemLink product-item')
hemisphere_title = hemisphere_soup.find_all('h3')
for title in hemisphere_title:
    hemisphere_title_clean.append(title.text.strip())
for hemisphere in all_hemispheres:
    url_component.append(hemisphere['href'])
url_component = list( dict.fromkeys(url_component))
#hemisphere_title_clean

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [26]:
hemisphere = hemisphere_url.split('/')[2]
image_urls = []
for url in url_component:
    hemisphere_url ='https://astropedia.'+hemisphere+'/download/'+url.split('/')[3]+'/'+url.split('/')[4]+'/'+url.split('/')[5]+'.tif/full.jpg'
    image_urls.append(hemisphere_url)   
image_urls       

In [27]:
image_urls

['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [36]:
hiu = {}
hiu.update({'title':hemisphere_title_clean,
            'img_url':image_urls})

In [37]:
hiu

{'title': ['Cerberus Hemisphere Enhanced',
  'Schiaparelli Hemisphere Enhanced',
  'Syrtis Major Hemisphere Enhanced',
  'Valles Marineris Hemisphere Enhanced'],
 'img_url': ['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']}

In [45]:
hemisphere_image_urls = []
for i in range (4):
    time.sleep(1)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    hemisphere_image_urls.append(dictionary)
    browser.back()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=80.0.3987.122)


In [25]:
hemisphere_image_urls

[]

In [21]:
hemisphere_image_urls = []
items = hemisphere_soup.find_all('div', class_='item')
hemispheres_main_url = 'https://astrogeology.usgs.gov' 

    # Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information  
    soup = BeautifulSoup( partial_img_html, 'html.parser')
        
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
        
    # Append the retreived information into a list of dictionaries 
    dictionary={"title":title,"img_url":img_url}
    hemisphere_image_urls.append(dictionary)

In [22]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]